In [1]:
import awkward as ak
ak.numba.register_and_check()
import cupy as cp
import cudf
import numpy as np
import awkward_pandas.cudf
import subprocess

def gpu_mem():
    print(subprocess.check_output("nvidia-smi | grep py", shell=True).split()[-2].decode())

ak.__version__, awkward_pandas.__version__

('2.6.3', '2023.8.1.dev25+g5dd12c5.d20240417')

In [5]:
df = cudf.read_parquet("/floppy/code/awkward/s.parquet")
gpu_mem()

188MiB


In [7]:
df.dtypes

a    list
dtype: object

In [8]:
df.iloc[0]  # each element is list-of-lists

a    [[1, 2, 3], [], [4, 5]]
Name: 0, dtype: list

In [9]:
# series accessor
df.a.ak

In [10]:
# allows all ak.* namespace, many identical to numpy equivalents
dir(df.a.ak)

['all',
 'almost_equal',
 'angle',
 'annotations',
 'any',
 'apply',
 'argcartesian',
 'argcombinations',
 'argmax',
 'argmin',
 'argsort',
 'array',
 'awkward',
 'backend',
 'behavior',
 'behaviors',
 'broadcast_arrays',
 'broadcast_fields',
 'builder',
 'cartesian',
 'categories',
 'combinations',
 'concatenate',
 'contents',
 'copy',
 'corr',
 'count',
 'count_nonzero',
 'covar',
 'cppyy',
 'drop_none',
 'enforce_type',
 'errors',
 'fields',
 'fill_none',
 'firsts',
 'flatten',
 'forms',
 'forth',
 'from_arrow',
 'from_arrow_schema',
 'from_avro_file',
 'from_buffers',
 'from_categorical',
 'from_cupy',
 'from_dlpack',
 'from_feather',
 'from_iter',
 'from_jax',
 'from_json',
 'from_numpy',
 'from_parquet',
 'from_rdataframe',
 'from_regular',
 'full_like',
 'highlevel',
 'imag',
 'index',
 'is_categorical',
 'is_none',
 'is_tuple',
 'is_valid',
 'isclose',
 'jax',
 'linear_fit',
 'local_index',
 'mask',
 'max',
 'mean',
 'merge_option_of_records',
 'merge_union_of_records',
 'metad

In [11]:
df.a.ak.sum(axis=None)

array(28000000)

In [27]:
df.a.ak.sum(axis=1)

AssertionError: CuPyKernel not found: ('awkward_ListOffsetArray_reduce_nonlocal_preparenext_64', <class 'numpy.int64'>, <class 'numpy.int64'>, <class 'numpy.int64'>, <class 'numpy.int64'>, <class 'numpy.int64'>, <class 'numpy.int64'>, <class 'numpy.int64'>)

See if this has been reported at https://github.com/scikit-hep/awkward/issues

In [12]:
# if output was array-like, it stays on the GPU
type(_)

cupy.ndarray

In [13]:
# fast reduction across three levels of nesting
%timeit df.a.ak.sum(axis=None)

7.09 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# ufunc maintains structure
np.negative(df.a.ak)

<Array [[[-1, -2, -3], ..., [-4, ...]], ...] type='2000000 * var * option[v...'>

In [15]:
gpu_mem()  # created new arrays on GPU, made new cuDF series

362MiB


In [23]:
# operator overload
print((df.a.ak + 1).head())

0    [[2, 3, 4], [], [5, 6]]
1    [[2, 3, 4], [], [5, 6]]
2    [[2, 3, 4], [], [5, 6]]
3    [[2, 3, 4], [], [5, 6]]
4    [[2, 3, 4], [], [5, 6]]
dtype: list
